In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
df =pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head()

#  **DATA PRE-PROCESSING**

In [ ]:
df['gender'].value_counts()

In [ ]:
df = df.drop(df[df['gender']== 'Other'].index)

In [ ]:
sns.countplot(x='gender' , data = df, palette="Set3")
xlabel='gender' 
ylabel='count'

In [ ]:
sns.boxplot(x='smoking_status', y='age' , data = df, palette="Set3")
xlabel='smoking_status'
ylabel='age'

In [ ]:
sns.boxplot(x='bmi', y='work_type' , data = df,palette="Set3")
xlabel='bmi'
ylabel='work_type'

In [ ]:
g = sns.catplot(x="gender", hue="work_type", col="stroke",
...                 data=df, kind="count",
...                 height=4, aspect=.7, palette="Set2");

In [ ]:
g = sns.catplot(x="gender", hue="ever_married", col="stroke",
...                 data=df, kind="count",
...                 height=4, aspect=.7, palette="Set2");

In [ ]:
g = sns.catplot(x="gender", hue="heart_disease", col="stroke",
...                 data=df, kind="count",
...                 height=4, aspect=.7, palette="husl")

In [ ]:
g = sns.catplot(x="gender", hue="Residence_type", col="stroke",
...                 data=df, kind="count",
...                 height=4, aspect=.7, palette="Set3")

In [ ]:
sns.countplot(x='smoking_status',data=df, hue= 'gender', palette= "Set3")
xlabel='Smoking status' 
ylabel='count'

In [ ]:
g = sns.catplot(x="gender", hue="smoking_status", col="stroke",
...                 data=df, kind="count",
...                 height=4, aspect=.7, palette="Set3")

In [ ]:
df.select_dtypes(['object']).columns

In [ ]:
dummies = pd.get_dummies(df[['gender','ever_married','work_type','Residence_type','smoking_status']],drop_first=True)
df = pd.concat([df,dummies],axis=1)
df = df.drop(['gender','ever_married','work_type','Residence_type','smoking_status'], axis=1)
df= df.drop(['id'], axis=1)

In [ ]:
df.columns

# CHECKING FOR NULL VALUES AND USING KNNImputer

In [ ]:
df.isnull().sum()

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
dataset = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)

In [ ]:
dataset.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(dataset.corr(), annot = True )
xlabel='' 
ylabel=''

# OVERSAMPLING MY DATASET DUE THE DATA IS UNEVEN

In [ ]:
X = dataset.drop('stroke',axis=1).values
y = dataset['stroke'].values

In [ ]:
from imblearn import under_sampling, over_sampling
print("Before OverSampling, counts of label '1': {}".format(sum(y == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(y == 0))) 
  
# import SMOTE module from imblearn library  

from imblearn.combine import SMOTEENN 
sm = SMOTEENN(random_state=10) 
X_over, y_over = sm.fit_resample(X, y) 
  
print('After OverSampling, the shape of train_X: {}'.format(X_over.shape)) 
print('After OverSampling, the shape of train_y: {} \n'.format(y_over.shape)) 
  
print("After OverSampling, counts of label '1': {}".format(sum(y_over == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(y_over == 0)))

# BEFORE START THE TRAINING I WILL NORMALIZE MY DATA

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.20, random_state=7)

In [ ]:
from sklearn import preprocessing
X_train_norm= preprocessing.normalize(X_train)
X_test_norm =preprocessing.normalize(X_test)

# COMPARTION OF ALGORITHMS

In [ ]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


# prepare configuration for cross validation test harness
seed = 7
#MODELS
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

In [ ]:
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    kfold = model_selection.KFold(n_splits=10)
    cv_results = model_selection.cross_val_score(model, X_train_norm, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

#  DECISION TREE CLASIFICATION

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train_norm,y_train)
predictions = dtree.predict(X_test_norm)

In [ ]:
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train_norm, y_train)
pred = neigh.predict(X_test_norm)

In [ ]:
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

#  MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=1000,activation = 'relu',solver='adam',random_state=1)
clf.fit(X_train_norm, y_train)
pred_mlp= clf.predict(X_test_norm)

In [ ]:
print(classification_report(y_test,pred_mlp))
print(confusion_matrix(y_test,pred_mlp))